In [ ]:
'''
Author: ChenHJ
Date: 2023-01-12 23:39:25
LastEditors: ChenHJ
LastEditTime: 2023-01-12 23:40:01
FilePath: /0302code/obs_CMIP_sIndR_ENSO_related.ipynb
Aim: 
01/12/2023:
用于计算ENSO-related的印度半岛南部降水和unrelated的印度半岛南部降水分别对应的降水，环流，在historical、ssp585时期
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm

# 读取数据

## 观测

In [ ]:
fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]
preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)
preGPCP_JJA_detrend = ca.detrend_dim(preGPCP_JJA.copy(), "time", deg=1, demean=False)

# preAIR = xr.open_dataarray("/home/ys17-23/Extension/All_India_Rainfall_index/AIR_mmperday.nc")
# preAIR_JJA = ca.p_time(preAIR, 6, 8, True)
# preAIR_JJA = preAIR_JJA.sel(time=(preAIR_JJA.time.dt.year>=1979) & (preAIR_JJA.time.dt.year <=2014))
# preAIR_JJA = ca.detrend_dim(preAIR_JJA, "time", deg=1, demean=False)

fpreCRU = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/cru_ts4.01_r144x72_195001-201412.nc")
preCRU = fpreCRU["pre"].sel(time=(fpreCRU["time"].dt.year>=1979) & (fpreCRU.time.dt.year<=2014))
preCRU_JJA = ca.p_time(preCRU, 6, 8, True)/30.67
preCRU_JJA.attrs["units"] = "mm/day"
preCRU_JJA = ca.detrend_dim(preCRU_JJA, "time", deg=1, demean=False)

fhgtERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc")
hgtERA5 = fhgtERA5["z"].sel(time=fhgtERA5["time"].dt.year>=1979)

fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"].sel(time=fuERA5["time"].dt.year>=1979)

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"].sel(time=fvERA5["time"].dt.year>=1979)

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"].sel(time=fspERA5["time"].dt.year>=1979)

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"].sel(time=fqERA5["time"].dt.year>=1979)

fwERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/omega_mon_r144x72_195001-201412.nc")
wERA5 = fwERA5["w"].sel(time=fwERA5["time"].dt.year>=1979)

fsstHad = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/HadISST_r144x72_1870-2020.nc")
sstHad = fsstHad["sst"].sel(time=(fsstHad["time"].dt.year>=1979) & (fsstHad["time"].dt.year<=2014))

hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True)
# hgtERA5_ver_JJA = hgtERA5_ver_JJA-hgtERA5_ver_JJA.mean(dim="lon", skipna=True)
hgtERA5_ver_JJA = ca.detrend_dim(hgtERA5_ver_JJA, "time", deg=1, demean=False)

uERA5_ver_MAM, uERA5_ver_JJA, uERA5_ver_SON, uERA5_ver_DJF = ca.mon_to_season(uERA5)
uERA5_ver_MAM = ca.detrend_dim(uERA5_ver_MAM, "time", deg=1, demean=False)
uERA5_ver_JJA = ca.detrend_dim(uERA5_ver_JJA, "time", deg=1, demean=False)
uERA5_ver_SON = ca.detrend_dim(uERA5_ver_SON, "time", deg=1, demean=False)
uERA5_ver_DJF = ca.detrend_dim(uERA5_ver_DJF, "time", deg=1, demean=False)

vERA5_ver_MAM, vERA5_ver_JJA, vERA5_ver_SON, vERA5_ver_DJF = ca.mon_to_season(vERA5)
vERA5_ver_MAM = ca.detrend_dim(vERA5_ver_MAM, "time", deg=1, demean=False)
vERA5_ver_JJA = ca.detrend_dim(vERA5_ver_JJA, "time", deg=1, demean=False)
vERA5_ver_SON = ca.detrend_dim(vERA5_ver_SON, "time", deg=1, demean=False)
vERA5_ver_DJF = ca.detrend_dim(vERA5_ver_DJF, "time", deg=1, demean=False)

qERA5_ver_JJA = ca.p_time(qERA5, 6, 9, True)
qERA5_ver_JJA = ca.detrend_dim(qERA5_ver_JJA, "time", deg=1, demean=False)

spERA5_MAM, spERA5_JJA, spERA5_SON, spERA5_DJF = ca.mon_to_season(spERA5)
spERA5_MAM = ca.detrend_dim(spERA5_MAM, "time", deg=1, demean=False)
spERA5_JJA = ca.detrend_dim(spERA5_JJA, "time", deg=1, demean=False)
spERA5_SON = ca.detrend_dim(spERA5_SON, "time", deg=1, demean=False)
spERA5_DJF = ca.detrend_dim(spERA5_DJF, "time", deg=1, demean=False)

wERA5_MAM, wERA5_JJA, wERA5_SON, wERA5_DJF = ca.mon_to_season(wERA5)
wERA5_MAM = ca.detrend_dim(wERA5_MAM, "time", deg=1, demean=False)
wERA5_JJA = ca.detrend_dim(wERA5_JJA, "time", deg=1, demean=False)
wERA5_SON = ca.detrend_dim(wERA5_SON, "time", deg=1, demean=False)
wERA5_DJF = ca.detrend_dim(wERA5_DJF, "time", deg=1, demean=False)
sstHad_MAM, sstHad_JJA, sstHad_SON, sstHad_DJF = ca.mon_to_season(sstHad)
sstHad_MAM = ca.detrend_dim(sstHad_MAM, "time", deg=1, demean=False)
sstHad_JJA = ca.detrend_dim(sstHad_JJA, "time", deg=1, demean=False)
sstHad_SON = ca.detrend_dim(sstHad_SON, "time", deg=1, demean=False)
sstHad_DJF = ca.detrend_dim(sstHad_DJF, "time", deg=1, demean=False)
ftERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/temp_mon_r144x72_195001-201412.nc")
tERA5 = ftERA5["t"].sel(time=ftERA5["time"].dt.year>=1979)
tERA5_ver_JJA = ca.p_time(tERA5, 6, 8, True)
tERA5_ver_JJA = ca.detrend_dim(tERA5_ver_JJA, "time", deg=1, demean=False)
divERA5_ver_JJA = ca.cal_divergence(uERA5_ver_JJA.sel(level=[200.0, 850.0]), vERA5_ver_JJA.sel(level=[200.0, 850.0]))
divERA5_ver_JJA = ca.detrend_dim(divERA5_ver_JJA, "time", deg=1, demean=False)

#   calculate the waver vapor vertical intergration
ptop = 1
g = 9.8
ERA5level = qERA5_ver_JJA.coords["level"].where(qERA5_ver_JJA.coords["level"] >= ptop) * 100.0
ERA5level.attrs["units"] = "Pa"
ERA5dp = geocat.comp.dpres_plevel(ERA5level, spERA5_JJA, ptop*100)
ERA5dpg = ERA5dp / g
ERA5dpg.attrs["units"] = "kg/m2"
# calculate the water vapor transport
integration_top = 200
uq_sl_ERA5_JJA = uERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
vq_sl_ERA5_JJA = vERA5_ver_JJA.loc[:, integration_top:, :, :] * qERA5_ver_JJA.loc[:, integration_top:, :, :].data * 1000.0
uq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
vq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
# calculate the whole levels water vapor transport
uqERA5_ver_JJA = (uq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
vqERA5_ver_JJA = (vq_sl_ERA5_JJA * ERA5dpg[:, list(ERA5level).index(integration_top*100):, :, :].data).sum(dim="level") / 1e05
uqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
vqERA5_ver_JJA.attrs["units"] = "100kg m-1 s-1"
uqERA5_ver_JJA = ca.detrend_dim(uqERA5_ver_JJA, "time", deg=1, demean=False)
vqERA5_ver_JJA = ca.detrend_dim(vqERA5_ver_JJA, "time", deg=1, demean=False)
# calculate the water vapor transportation divergence
uqvqdivERA5_ver_JJA = ca.cal_divergence(uqERA5_ver_JJA, vqERA5_ver_JJA)
uqvqdivERA5_ver_JJA = ca.detrend_dim(uqvqdivERA5_ver_JJA, "time", deg=1, demean=False)

## historical

In [ ]:
fprehis = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/pr_historical_r144x72_197901-201412.nc")
prehis_JJA = fprehis["pr"].sel(time=fprehis["time"].dt.year>=1979)
prehis_JJA.attrs["units"] = "mm/day"
prehis_JJA.attrs["standard_name"] = "precipitation"

fhgthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/zg_historical_r144x72_197901-201412.nc")
hgthis_ver_JJA = fhgthis_ver_JJA["zg"].sel(time=fhgthis_ver_JJA["time"].dt.year>=1979)
# hgthis_ver_JJA = hgthis_ver_JJA - hgthis_ver_JJA.mean(dim="lon", skipna=True)

fuhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ua_historical_r144x72_197901-201412.nc")
uhis_ver_JJA = fuhis_ver_JJA["ua"].sel(time=fuhis_ver_JJA["time"].dt.year>=1979)

fvhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/va_historical_r144x72_197901-201412.nc")
vhis_ver_JJA = fvhis_ver_JJA["va"].sel(time=fvhis_ver_JJA["time"].dt.year>=1979)

# fwhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/wap_historical_r144x72_197901-201412.nc") 
# whis_ver_JJA = fwhis_ver_JJA["wap"].sel(time=fwhis_ver_JJA["time"].dt.year>=1979)

fssthis_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_MAM = fssthis_MAM["__xarray_dataarray_variable__"].sel(time=fssthis_MAM["time"].dt.year>=1979)

fssthis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_JJA = fssthis_JJA["__xarray_dataarray_variable__"].sel(time=fssthis_JJA["time"].dt.year>=1979)

fssthis_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_SON = fssthis_SON["__xarray_dataarray_variable__"].sel(time=fssthis_SON["time"].dt.year>=1979)

fssthis_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/tos_historical_r144x72_197901-201412.nc")
ssthis_DJF = fssthis_DJF["__xarray_dataarray_variable__"].sel(time=fssthis_DJF["time"].dt.year>=1979)

fuhis_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/u_historical_r144x72_197901-201412.nc")
uhis_MAM = fuhis_MAM["u"].sel(time=fuhis_MAM["time"].dt.year>=1979)

fuhis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/u_historical_r144x72_197901-201412.nc")
uhis_JJA = fuhis_JJA["u"].sel(time=fuhis_JJA["time"].dt.year>=1979)

fuhis_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/u_historical_r144x72_197901-201412.nc")
uhis_SON = fuhis_SON["u"].sel(time=fuhis_SON["time"].dt.year>=1979)

fuhis_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/u_historical_r144x72_197901-201412.nc")
uhis_DJF = fuhis_DJF["u"].sel(time=fuhis_DJF["time"].dt.year>=1979)

fvhis_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/v_historical_r144x72_197901-201412.nc")
vhis_MAM = fvhis_MAM["v"].sel(time=fvhis_MAM["time"].dt.year>=1979)

fvhis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/v_historical_r144x72_197901-201412.nc")
vhis_JJA = fvhis_JJA["v"].sel(time=fvhis_JJA["time"].dt.year>=1979)

fvhis_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/v_historical_r144x72_197901-201412.nc")
vhis_SON = fvhis_SON["v"].sel(time=fvhis_SON["time"].dt.year>=1979)

fvhis_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/v_historical_r144x72_197901-201412.nc")
vhis_DJF = fvhis_DJF["v"].sel(time=fvhis_DJF["time"].dt.year>=1979)

fthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ta_historical_r144x72_197901-201412.nc")
this_ver_JJA = fthis_ver_JJA["ta"].sel(time=fthis_ver_JJA["time"].dt.year>=1979)

sphis_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/sp_historical_r144x72_197901-201412.nc")
sphis_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/sp_historical_r144x72_197901-201412.nc")
sphis_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/sp_historical_r144x72_197901-201412.nc")
sphis_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/sp_historical_r144x72_197901-201412.nc")

fuqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_uq_dpg.nc")
uqhis_ver_JJA = fuqhis_ver_JJA["uq_dpg"].sel(time=fuqhis_ver_JJA["time"].dt.year>=1979)

fvqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_vq_dpg.nc")
vqhis_ver_JJA = fvqhis_ver_JJA["vq_dpg"].sel(time=fvqhis_ver_JJA["time"].dt.year>=1979)

fuqvqdivhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_div_uqvq_dpg.nc")
uqvqdivhis_ver_JJA = fuqvqdivhis_ver_JJA["div_uqvq"].sel(time=fuqvqdivhis_ver_JJA["time"].dt.year>=1979)

fsphis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ps_historical_r144x72_197901-201412.nc")
sphis_ver_JJA = fsphis_ver_JJA["ps"].sel(time=fsphis_ver_JJA["time"].dt.year>=1979)

divhis_ver_JJA = ca.cal_divergence(uhis_ver_JJA.sel(level=[200.0, 850.0]), vhis_ver_JJA.sel(level=[200.0, 850.0]))

## ssp585

In [ ]:
fpressp585_p3 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/pr_ssp585_p3_r144x72_206401-209912.nc")
pressp585_p3_JJA = fpressp585_p3["pr"].sel(time=fpressp585_p3.time.dt.year>=2064)
pressp585_p3_JJA.attrs["units"] = "mm/day"
pressp585_p3_JJA.attrs["standard_name"] = "precipitation"

fhgtssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/zg_ssp585_p3_r144x72_206401-209912.nc")
hgtssp585_p3_ver_JJA = fhgtssp585_p3_ver_JJA["zg"].sel(time=fhgtssp585_p3_ver_JJA.time.dt.year>=2064)
# hgtssp585_p3_ver_JJA = hgtssp585_p3_ver_JJA - hgtssp585_p3_ver_JJA.mean(dim="lon", skipna=True)


fussp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ua_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_ver_JJA = fussp585_p3_ver_JJA["ua"].sel(time=fussp585_p3_ver_JJA.time.dt.year>=2064)

fvssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/va_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_ver_JJA = fvssp585_p3_ver_JJA["va"].sel(time=fvssp585_p3_ver_JJA.time.dt.year>=2064)

# fwssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
# wssp585_p3_ver_JJA = fwssp585_p3_ver_JJA["wap"].sel(time=fwssp585_p3_ver_JJA.time.dt.year>=2064)

fsstssp585_p3_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_MAM = fsstssp585_p3_MAM["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_MAM.time.dt.year>=2064)

fsstssp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_JJA = fsstssp585_p3_JJA["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_JJA.time.dt.year>=2064)

fsstssp585_p3_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_SON = fsstssp585_p3_SON["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_SON.time.dt.year>=2064)

fsstssp585_p3_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
sstssp585_p3_DJF = fsstssp585_p3_DJF["__xarray_dataarray_variable__"].sel(time=fsstssp585_p3_DJF.time.dt.year>=2064)

fussp585_p3_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_MAM = fussp585_p3_MAM["u"].sel(time=fussp585_p3_MAM.time.dt.year>=2064)

fussp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_JJA = fussp585_p3_JJA["u"].sel(time=fussp585_p3_JJA.time.dt.year>=2064)

fussp585_p3_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_SON = fussp585_p3_SON["u"].sel(time=fussp585_p3_SON.time.dt.year>=2064)

fussp585_p3_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/u_ssp585_p3_r144x72_206401-209912.nc")
ussp585_p3_DJF = fussp585_p3_DJF["u"].sel(time=fussp585_p3_DJF.time.dt.year>=2064)

fvssp585_p3_MAM = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_MAM = fvssp585_p3_MAM["v"].sel(time=fvssp585_p3_MAM.time.dt.year>=2064)

fvssp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_JJA = fvssp585_p3_JJA["v"].sel(time=fvssp585_p3_JJA.time.dt.year>=2064)

fvssp585_p3_SON = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_SON = fvssp585_p3_SON["v"].sel(time=fvssp585_p3_SON.time.dt.year>=2064)

fvssp585_p3_DJF = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/v_ssp585_p3_r144x72_206401-209912.nc")
vssp585_p3_DJF = fvssp585_p3_DJF["v"].sel(time=fvssp585_p3_DJF.time.dt.year>=2064)

ftssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ta_ssp585_p3_r144x72_206401-209912.nc")
tssp585_p3_ver_JJA = ftssp585_p3_ver_JJA["ta"].sel(time=ftssp585_p3_ver_JJA.time.dt.year>=2064)

divssp585_p3_ver_JJA = ca.cal_divergence(ussp585_p3_ver_JJA.sel(level=[200.0, 850.0]), vssp585_p3_ver_JJA.sel(level=[200.0, 850.0]))

fuqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_uq_dpg.nc")
uqssp585_p3_ver_JJA = fuqssp585_p3_ver_JJA["uq_dpg"].sel(time=fuqssp585_p3_ver_JJA["time"].dt.year>=2064)

fvqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_vq_dpg.nc")
vqssp585_p3_ver_JJA = fvqssp585_p3_ver_JJA["vq_dpg"].sel(time=fvqssp585_p3_ver_JJA["time"].dt.year>=2064)

fuqvqdivssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_p3_div_uqvq_dpg.nc")
uqvqdivssp585_p3_ver_JJA = fuqvqdivssp585_p3_ver_JJA["div_uqvq"].sel(time=fuqvqdivssp585_p3_ver_JJA["time"].dt.year>=2064)

fspssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ps_ssp585_p3_r144x72_206401-209912.nc")
spssp585_p3_ver_JJA = fspssp585_p3_ver_JJA["ps"].sel(time=fspssp585_p3_ver_JJA["time"].dt.year>=2064)

# 计算Nino3.4 index并去趋势

In [ ]:
# 计算Nino 3.4 index并去趋势
Nino34Had = ca.cal_area_weighted_mean(ca.filplonlat(sstHad.copy()).loc[:, -5:5, -170:-120])
Nino34Had = ca.detrend_dim(Nino34Had, "time", deg=1, demean=False)
Nino34Had_MAM, Nino34Had_JJA, Nino34Had_SON, Nino34Had_DJF = ca.mon_to_season(Nino34Had)
Nino34Had_MAM = ca.detrend_dim(Nino34Had_MAM, "time", deg=1, demean=False)
Nino34Had_JJA = ca.detrend_dim(Nino34Had_JJA, "time", deg=1, demean=False)
Nino34Had_SON = ca.detrend_dim(Nino34Had_SON, "time", deg=1, demean=False)
Nino34Had_DJF = ca.detrend_dim(Nino34Had_DJF, "time", deg=1, demean=False)
Nino34Had_MAM_nor = ca.standardize(Nino34Had_MAM)
Nino34Had_JJA_nor = ca.standardize(Nino34Had_JJA)
Nino34Had_SON_nor = ca.standardize(Nino34Had_SON)
Nino34Had_DJF_nor = ca.standardize(Nino34Had_DJF)

In [ ]:
# 计算Nino3.4 index并去趋势
ssthis_nondetrend = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/non_detrend/tos_historical_r144x72_195001-201412.nc")
sstssp585_nondetrend = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/non_detrend/tos_ssp585_r144x72_201501-209912.nc")

Nino34CMIP6_MAM, Nino34CMIP6_JJA, Nino34CMIP6_SON, Nino34CMIP6_DJF = ca.mon_to_season(ca.cal_area_weighted_mean(ca.filplonlat(xr.concat([ssthis_nondetrend.sel(time=ssthis_nondetrend.time.dt.year>=1979), sstssp585_nondetrend], "time")).loc[:,:,-5:5,-170:-120]))

Nino34CMIP6_MAM_detrend = ca.detrend_dim(Nino34CMIP6_MAM, "time", deg=1, demean=True)
Nino34CMIP6_JJA_detrend = ca.detrend_dim(Nino34CMIP6_JJA, "time", deg=1, demean=True)
Nino34CMIP6_SON_detrend = ca.detrend_dim(Nino34CMIP6_SON, "time", deg=1, demean=True)
Nino34CMIP6_DJF_detrend = ca.detrend_dim(Nino34CMIP6_DJF, "time", deg=1, demean=True)
Nino34his_MAM = ca.detrend_dim(Nino34CMIP6_MAM_detrend.sel(time=Nino34CMIP6_MAM_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_JJA = ca.detrend_dim(Nino34CMIP6_JJA_detrend.sel(time=Nino34CMIP6_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_SON = ca.detrend_dim(Nino34CMIP6_SON_detrend.sel(time=Nino34CMIP6_SON_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
Nino34his_DJF = ca.detrend_dim(Nino34CMIP6_DJF_detrend.sel(time=Nino34CMIP6_DJF_detrend.time.dt.year<=2014), "time", deg=1, demean=True)

Nino34ssp585_p3_MAM = ca.detrend_dim(Nino34CMIP6_MAM_detrend.sel(time=Nino34CMIP6_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_JJA = ca.detrend_dim(Nino34CMIP6_JJA_detrend.sel(time=Nino34CMIP6_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_SON = ca.detrend_dim(Nino34CMIP6_SON_detrend.sel(time=Nino34CMIP6_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
Nino34ssp585_p3_DJF = ca.detrend_dim(Nino34CMIP6_DJF_detrend.sel(time=Nino34CMIP6_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

Nino34his_MAM_nor = ca.standardize(Nino34his_MAM.copy())
Nino34his_JJA_nor = ca.standardize(Nino34his_JJA.copy())
Nino34his_SON_nor = ca.standardize(Nino34his_SON.copy())
Nino34his_DJF_nor = ca.standardize(Nino34his_DJF.copy())

Nino34ssp585_p3_MAM_nor = ca.standardize(Nino34ssp585_p3_MAM.copy())
Nino34ssp585_p3_JJA_nor = ca.standardize(Nino34ssp585_p3_JJA.copy())
Nino34ssp585_p3_SON_nor = ca.standardize(Nino34ssp585_p3_SON.copy())
Nino34ssp585_p3_DJF_nor = ca.standardize(Nino34ssp585_p3_DJF.copy())

# 计算印度南部降水

In [ ]:
lat = prehis_JJA.coords["lat"]
lon = prehis_JJA.coords["lon"]
# 南部
sIndia_N = 20.0
sIndia_S = 5.0
sIndia_W = 70.0
sIndia_E = 85.0
lat_sIndia_range = lat[(lat >= sIndia_S) & (lat <= sIndia_N)]
lon_sIndia_range = lon[(lon >= sIndia_W) & (lon <= sIndia_E)]

preGPCP_sIndia_JJA = ca.cal_area_weighted_mean(preGPCP_JJA_detrend.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
prehis_sIndia_JJA = ca.cal_area_weighted_mean(prehis_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))
pressp585_p3_sIndia_JJA = ca.cal_area_weighted_mean(pressp585_p3_JJA.sel(lat=lat_sIndia_range, lon=lon_sIndia_range))

# detrend
preGPCP_sIndia_JJA_detrend = ca.detrend_dim(preGPCP_sIndia_JJA.copy(), "time", deg=1, demean=True)
prehis_sIndia_JJA_detrend = ca.detrend_dim(prehis_sIndia_JJA.copy(), "time", deg=1, demean=True)
pressp585_p3_sIndia_JJA_detrend = ca.detrend_dim(pressp585_p3_sIndia_JJA.copy(), "time", deg=1, demean=True)

# standardize
preGPCP_sIndia_JJA_nor = ca.standardize(preGPCP_sIndia_JJA_detrend)
prehis_sIndia_JJA_nor = ca.standardize(prehis_sIndia_JJA_detrend)
pressp585_p3_sIndia_JJA_nor = ca.standardize(pressp585_p3_sIndia_JJA_detrend)

# 分离与前冬Nino3.4 index相关或无关的印度南部降水

In [ ]:
tmp = preGPCP_sIndia_JJA_nor[1:].copy()
tmp.coords["time"] = Nino34Had_DJF_nor.time
linre_DJFnino34_sIndR = stats.linregress(Nino34Had_DJF_nor, tmp)
sIndR_GPCP_ENSO_related = linre_DJFnino34_sIndR[0]*Nino34Had_DJF_nor+linre_DJFnino34_sIndR[1]
sIndR_GPCP_ENSO_unrelated = tmp - sIndR_GPCP_ENSO_related

In [ ]:
tmp = prehis_sIndia_JJA_nor[:,1:].copy()
tmp.coords["time"] = Nino34his_DJF_nor.time
linre_his_DJFnino34_sIndR = ca.dim_linregress(Nino34his_DJF_nor, tmp)
sIndR_his_ENSO_related = linre_his_DJFnino34_sIndR[:][0]*Nino34his_DJF_nor+linre_his_DJFnino34_sIndR[:][1]
sIndR_his_ENSO_unrelated = tmp - sIndR_his_ENSO_related

In [ ]:
tmp = pressp585_p3_sIndia_JJA_nor[:,1:].copy()
tmp.coords["time"] = Nino34ssp585_p3_DJF_nor[:,:-1].time
linre_ssp585_p3_DJFnino34_sIndR = ca.dim_linregress(Nino34ssp585_p3_DJF_nor[:,:-1], tmp)
sIndR_ssp585_p3_ENSO_related = linre_ssp585_p3_DJFnino34_sIndR[:][0]*Nino34ssp585_p3_DJF_nor+linre_ssp585_p3_DJFnino34_sIndR[:][1]
sIndR_ssp585_p3_ENSO_unrelated = tmp - sIndR_ssp585_p3_ENSO_related